## Chapter 5. 오차역전파법(backpropagation)

### 5.1 계산 그래프(computational graph)

+ 계산 과정을 그래프로 나타낸 것
+ 복수의 노트(node)와 에지(edge)로 표현되고, 노드 사이의 직선을 '에지'라고 함

#### 5.1.1 계산 그래프로 풀다

1. 계산 그래프를 구성한다.
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다. (순전파, forward propagation)

#### 5.1.2 국소적 계산

+ 자신과 직접 관계된 작은 범위
+ 각 노드는 자신과 관련된 계산외에는 복잡한 계산이더라도 아무런 상관이 없다.

### 5.2 연쇄법칙(chain rule)

#### 5.2.2 연쇄법칙이란?

+ 합성함수 : 여러 함수로 구성된 함수
+ 합성 함수의 미분에 대한 성질이며, '합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다'로 정의된다.

### 5.3 역전파

#### 5.3.1 덧셈 노드의 역전파
+ 덧셈 노드의 역전파는 1을 곱하기만 할 뿐이므로, 상류에서 전해진 값을 하류로 전달한다.

#### 5.3.2 곱셈 노드의 역전파
+ 곱셈 노드의 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값'을 곱해서 하류로 전달한다.

### 5.4 단순한 계층 구현하기

#### 5.4.1 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out
    
    def backward(self, dout):
        dx = dout * self.y # x 와 y 를 바꾼다.
        dy = dout * self.x

        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


#### 5.4.2 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


### 5.5 활성화 함수 계층 구현하기

#### 5.5.1 ReLU 계층

In [6]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


In [8]:
import numpy as np

x = np.array( [[1.0, -0.5], [-2.0, 3.0]] )
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [9]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


#### 5.5.2 Sigmoid 계층

In [15]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

### 5.6 Affine/Softmax 계층 구현하기

#### 5.6.2 배치용 Affine 계층
+ 신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서 'Affine Transformation' 이라고 한다.

In [16]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

#### 5.6.3 Softmax-with-Loss 계층

In [17]:
def softmax(a) :
    c = np.max(a)
    exp_a = np.exp(a - c) # overflow 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

def cross_entropy_error(y, t):
    if y.ndim == 1: # np.array(y)의 차원(dimention)
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return - np.sum(t * np.log(y + 1e-7)) / batch_size

In [18]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None    # Softmax의 출력
        self.t = None    # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

### 5.7 오차역전파법 구현하기